### In this classwork we will 
a) show how to load image data with Pytorch. Then we will 

b) build and train a feed forward and a convolutional neural network to classify images as Cat Vs No-Cat, 

c) learn how to use a validation set to evaluate the performance during training 

d) use regularization techniques to suppress overfitting 

e) save a trained model to infer on a testin dataset.

Make the necessary imports

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import h5py

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms

# a)

Load the data from an hdf5 file

In [ ]:
train_file = h5py.File('train_catvnoncat.h5', 'r')
test_file = h5py.File('test_catvnoncat.h5', 'r')


Inspect how imbalanced the dataset is in terms of labels

In [ ]:
u, c = np.unique(train_file['train_set_y'][:]==1, return_counts=True)
print(u,c)
u, c = np.unique(test_file['test_set_y'][:]==1, return_counts=True)
print(u,c)

In Pytorch we typically need a Dataset that is fed into a Dataloader. Then we iterate over the Dataloader object 

In [ ]:
class dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.int64)

    def __getitem__(self, idx):
        return self.x[idx].permute(2, 0, 1), self.y[idx]
    def __len__(self):
        length = self.x.shape[0]
        return length

In [ ]:
train_set = dataset(train_file['train_set_x'][:], train_file['train_set_y'][:])

In [ ]:
train_loader = torch.utils.data.DataLoader(
            train_set,
            batch_size=8,
            shuffle=True,
            drop_last = True,
        ) 

Let's now inspect the dataloader

In [ ]:
print('Iterating the dataloader')
e = iter(train_loader)
print('Gives us a list of length', len(e))
print('Which is the number of batches of data')
e.next()
samples = e.next()
print('By taking one batch, we get a list of:', len(samples))
print('with two tensors: 0th', samples[0].shape, '1st', samples[1].shape)
cats = samples[1][:] == 1
#plt.imshow(samples[0][5])
idx_cats = np.where(cats==True)
idx_nocat = np.where(cats==False)
for c in idx_cats[0]:
    plt.imshow(samples[0][c].to(dtype=torch.uint8).permute(1, 2, 0))
    plt.show()
    print('With label:', samples[1][c])
for d in idx_nocat[0]:
    plt.imshow(samples[0][d].to(dtype=torch.uint8).permute(1, 2, 0))
    plt.show()
    print('With label:', samples[1][d])

# b)

Let's first build a simple Feed Forward Neural Network 

In [ ]:
class SimpleFeedForward(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleFeedForward, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)        
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        
        
    def forward(self, x):
        x = self.l1(x)
        x = F.relu(x)
        #x = self.l2(x)
        #x = F.relu(x)        
        out = self.l3(x)
        
        return out

In [ ]:
npixels = 64
input_size = (npixels)**2 
hidden = 256
num_classes =2
epochs = 20
lr = 10e-3

# If you want to use a GPU 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = SimpleFeedForward(input_size, hidden, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
#steps = len(train_loader)

ff_loss_history = []
ff_acc_history = []

for epoch in range(epochs):
    
    total_loss = 0
    
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        
        #For accuracy
        n_correct = 0
        n_samples = 0 
        
        images = images.to(device) #(B, C, H, W)  
        labels = labels.to(device)
        
        #convert images grey for FF network
        images = torch.mean(images, dim=1) # (B, H, W)
        images = images.reshape(-1, input_size) #(B, H*W)
        
        # here you predict
        y_pred = model(images)
        loss = criterion(y_pred, 
                         labels)
        total_loss += loss
        
        _, predictions = torch.max(y_pred, 1)
            
        #For accuracy
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        # Zeros the gradients
        optimizer.zero_grad()
        # Calculate the gradients
        loss.backward()
        #Update the weights
        optimizer.step()
        
    acc = 100.0 * n_correct / n_samples    
    ff_loss_history.append(total_loss/i)
    ff_acc_history.append(acc)
    print(f'epoch:{epoch:.0f}|loss:{ff_loss_history[-1]:.2f}|accuracy = {acc:.2f}')
    
    

   

# Now let's build a Convolutional Network

In [ ]:
class SimpleConv(nn.Module):
    def __init__(self):
        super(SimpleConv, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 2)        
        self.conv2 = nn.Conv2d(8, 16, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*15*15, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        #print(x.shape)
        x = self.pool(F.relu(self.conv1(x))) # (B, C , H, W)
        #print(x.shape)
        x = self.pool(F.relu(self.conv2(x))) # (B, C , H, W)
        #print(x.shape)
        x = x.view(-1, 16*15*15) # (B, C*H*W)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        #print(x.shape)
        x = F.relu(self.fc2(x))
        #print(x.shape)
        x = self.fc3(x)
        #print(x.shape)
        
        return x
        
    
        
    

In [ ]:
model = SimpleConv().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:

epochs = 20

conv_loss_history = []
conv_acc_history = []

for epoch in range(epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        n_correct = 0
        n_samples = 0 
        
        images = images.to(device) #(B, C, H, W)
        labels = labels.to(device)
        
        y_pred = # YOUR CODE HERE 
        loss = # YOUR CODE HERE 
        _, predictions = # YOUR CODE HERE 
            
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        optimizer.# YOUR CODE HERE 
        loss.# YOUR CODE HERE 
        optimizer.# YOUR CODE HERE 
        
    acc = 100.0 * n_correct / n_samples    
    conv_loss_history.append(loss)
    conv_acc_history.append(acc)
    print(f'epoch:{epoch:.0f}|loss:{loss.item():.2f}|accuracy = {acc:.2f}')
        

Now let's plot the results for both  the Feed forward and the Convolutional network

In [ ]:
plt.plot(np.arange(epochs), ff_loss_history, label='Feed Forward')
plt.plot(np.arange(epochs), conv_loss_history, label='Convolutional')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(epochs), ff_acc_history, label='Feed Forward')
plt.plot(np.arange(epochs), conv_acc_history, label='Convolutional')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Now let's do the same for the Fashion-MNIST dataset. The Cat no-Cat datasets is an easy one to classify but quite small for the results to be trusted.

In [ ]:
#Load or download the FashionMNIST dataset

train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()])) 

In [ ]:
train_loader = # YOUR CODE HERE (
            train_set,
            batch_size=128,
            shuffle=True,
            drop_last = True,
        ) 

Let's inpect the dataloader again

In [ ]:
print('Iterating the dataloader')
e = iter(train_loader)
print('Gives us a list of length', len(e))
print('Which is the number of batches of data')
e.next()
samples = e.next()
print('By taking one batch, we get a list of:', len(samples))
print('with two tensors: 0th', samples[0].shape, '1st', samples[1].shape)
for i, c in enumerate(samples[0]):
    plt.imshow(torch.squeeze(c))
    plt.show()
    print('With label:', samples[1][i])


In [ ]:
npixels = 28
input_size = (npixels)**2 
hidden = 256
num_classes =10
epochs = 10
lr = 10e-3

# If you want to use a GPU 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = SimpleFeedForward(input_size, hidden, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
#steps = len(train_loader)

ff_loss_history = []
ff_acc_history = []

for epoch in range(epochs):
    
    total_loss = 0
    
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        
        #For accuracy
        n_correct = 0
        n_samples = 0 
        
        images = images.to(device) #(B, C, H, W)  
        labels = labels.to(device)
        
        #convert images grey for FF network
        images = torch.mean(images, dim=1) # (B, H, W)
        images = images.reshape(-1, input_size) #(B, H*W)
        
        # here you predict
        y_pred = model(images)
        loss = criterion(y_pred, 
                         labels)
        total_loss += loss
        
        _, predictions = torch.max(y_pred, 1)
            
        #For accuracy
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        # Zeros the gradients
        optimizer.zero_grad()
        # Calculate the gradients
        loss.backward()
        #Update the weights
        optimizer.step()
        
    acc = 100.0 * n_correct / n_samples    
    ff_loss_history.append(total_loss/i)
    ff_acc_history.append(acc)
    print(f'epoch:{epoch:.0f}|loss:{ff_loss_history[-1]:.2f}|accuracy = {acc:.2f}')
    
    

   

In [ ]:
class SimpleConv(nn.Module):
    def __init__(self):
        super(SimpleConv, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 2)        
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*13*13, 128)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        #print(x.shape)
        x = self.pool(F.relu(self.conv1(x))) # (B, C, H, W)
        #print(x.shape)
        x = x.view(-1, 16*13*13) # (B, C*H*W)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        #print(x.shape)
        x = self.fc2(x)
        #print(x.shape)
        
        return x
    

In [ ]:
model = # YOUR CODE HERE
criterion = nn.# YOUR CODE HERE
optimizer = torch.# YOUR CODE HERE

In [ ]:
epochs = 10

conv_loss_history = []
conv_acc_history = []

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        n_correct = 0
        n_samples = 0 
        
        images = images.to(device) #(B, C, H, W)
        labels = labels.to(device)
        
        y_pred = model(images)
        loss = criterion(y_pred, labels)
        total_loss += loss
        
        _, predictions = torch.max(y_pred, 1)
            
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    acc = 100.0 * n_correct / n_samples    
    conv_loss_history.append(total_loss/i)
    conv_acc_history.append(acc)
    print(f'epoch:{epoch:.0f}|loss:{conv_loss_history[-1]:.2f}|accuracy = {acc:.2f}')
        

In [ ]:
plt.plot(np.arange(epochs), ff_loss_history, label='Feed Forward')
plt.plot(np.arange(epochs), conv_loss_history, label='Convolutional')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(epochs), ff_acc_history, label='Feed Forward')
plt.plot(np.arange(epochs), conv_acc_history, label='Convolutional')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# c)

# Now let's see how a validation set can be used to monitor the overfitting of a model during training. We will stick to the convolutional network

In [ ]:
train_set, val_set = torch.utils.data.random_split(
    train_set,
    [int(len(train_set)*0.8), int(len(train_set)*0.2)]
)

In [ ]:
train_loader = torch.utils.data.DataLoader(
            # YOUR CODE HERE,
            batch_size=128,
            shuffle=True,
            drop_last = True,
        ) 

val_loader = torch.utils.data.DataLoader(
            # YOUR CODE HERE,
            batch_size=128,
            shuffle=True,
            drop_last = True,
        ) 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleConv().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:

train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []


for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        train_correct = 0
        train_samples = 0 

        
        images = images.to(device) #(B, C, H, W)
        labels = labels.to(device)
        
        y_pred = model(images)
        train_loss = criterion(y_pred, labels)
        total_loss += train_loss
        
        _, predictions = torch.max(y_pred, 1)
            
        train_samples += labels.shape[0]
        train_correct += (predictions == labels).sum().item()
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
    
    train_acc = 100.0 * train_correct / train_samples
    train_loss_history.append(total_loss/i)
    train_acc_history.append(train_acc)    

    # Now do the validation
    model.eval()
    total_loss=0
    with torch.no_grad():
        val_correct = 0
        val_samples = 0 
        for i, (images, labels) in enumerate(val_loader):
            images = images.to(device) #(B, C, H, W)
            labels = labels.to(device)
            
            y_pred = # YOUR CODE HERE 
            val_loss = # YOUR CODE HERE
            total_loss += val_loss
            _, predictions = torch.max(y_pred, 1)
            
            val_samples += labels.shape[0]
            val_correct += (predictions == labels).sum().item()
        val_acc = 100.0 * val_correct / val_samples
        val_acc_history.append(val_acc)
        val_loss_history.append(total_loss/i)
        
        print(f'Epoch= {epoch:.0f}')
        print(f'|Training|Loss:{train_loss_history[-1]:.2f}|Acc:{train_acc:.2f}|')
        print(f'|Validation|Loss:{val_loss_history[-1]:.2f}|Acc:{val_acc:.2f}|')

del model

In [ ]:
plt.plot(np.arange(epochs), train_loss_history, label='Training')
plt.plot(np.arange(epochs), val_loss_history, label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(epochs), train_acc_history, label='Training')
plt.plot(np.arange(epochs), val_acc_history, label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Now let's modify the existing convolutional network adding some regularization. 
Dropout, batch norm, weight initialization, L2 regulatization 

In [ ]:
class AdvanceConv(nn.Module):
    def __init__(self):
        super(AdvanceConv, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 3, 2)
        self.fc1 = nn.Linear(16*13*13, 128)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(128, num_classes)
        
        self.dropout = nn.Dropout(0.5)
        self.bn =nn.BatchNorm1d(128)       
        
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Linear, nn.Conv2d)):
                nn.init.xavier_normal_(m.weight.data)
                m.bias.data.fill_(0.1)
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    
    def forward(self, x):
        #print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        #print(x.shape)
        x = x.view(-1, 16*13*13)
        #print(x.shape)
        x = F.relu((self.fc1(x)))
        x = self.dropout(x)
        #print(x.shape)
        x = self.fc3(x)
        #print(x.shape)
        
        return x
        
    
        
    




In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AdvanceConv().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=10e-3)

In [ ]:
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []


epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for i, (images, labels) in enumerate(train_loader):
    
        train_correct = 0
        train_samples = 0 
    
        images = images.to(device) #(B, C, H, W)
        labels = labels.to(device)
        
        y_pred = model(images)
        train_loss = criterion(y_pred, labels)
        total_loss += train_loss
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        _, predictions = torch.max(y_pred, 1)
            
        train_samples += labels.shape[0]
        train_correct += (predictions == labels).sum().item()
        
    
    train_acc = 100.0 * train_correct / train_samples
    train_loss_history.append(total_loss/i)
    train_acc_history.append(train_acc)    
    
    total_loss =0
    with torch.no_grad():
        val_correct = 0
        val_samples = 0 
        for i, (images, labels) in enumerate(val_loader):
            images = images.to(device) #(B, C, H, W)
            labels = labels.to(device)
            
            y_pred = model(images)
            val_loss = criterion(y_pred, labels)
            total_loss += val_loss
            _, predictions = torch.max(y_pred, 1)
            
            val_samples += labels.shape[0]
            val_correct += (predictions == labels).sum().item()
        
        val_acc = 100.0 * val_correct / val_samples
        val_acc_history.append(val_acc)
        val_loss_history.append(total_loss/i)
        
    print(f'Epoch= {epoch:.0f}')
    print(f'|Training|Loss:{train_loss_history[-1]:.2f}|Acc:{train_acc:.2f}|')
    print(f'|Validation|Loss:{val_loss_history[-1]:.2f}|Acc:{val_acc:.2f}|')
                         
torch.save(model.state_dict(), 'best_model.pt')           
del model

In [ ]:
plt.plot(np.arange(epochs), train_loss_history, label='Training')
plt.plot(np.arange(epochs), val_loss_history, label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(epochs), train_acc_history, label='Training')
plt.plot(np.arange(epochs), val_acc_history, label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Inference

In [ ]:
model = AdvanceConv().to(device)
model.load_state_dict(torch.load('best_model.pt'))
model.eval()

In [ ]:
#test_set = dataset(test_file['test_set_x'][:], test_file['test_set_y'][:])

test_loader = torch.utils.data.DataLoader(
            test_set,
            batch_size=1,
            shuffle=False,
            drop_last = False,
        ) 

In [ ]:
with torch.no_grad():
    test_correct = 0
    test_samples = 0 
    total_loss = 0
    for i, (images, labels) in enumerate(test_loader):
        
        images = images.to(device) #(B, C, H, W)
        labels = labels.to(device)

        y_pred = model(images)
        test_loss = criterion(y_pred, labels)
        total_loss += test_loss
        _, predictions = torch.max(y_pred, 1)

        test_samples += labels.shape[0]
        test_correct += (predictions == labels).sum().item()
    
    test_acc = 100.0 * val_correct / val_samples
    print(i)
    total_loss = total_loss/i

    print('Inference')
    print(f'|Testing|Loss:{total_loss:.2f}|Acc:{test_acc:.2f}|')
    